In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rocketpy import Function, LiquidMotor, MassFlowRateBasedTank, UllageBasedTank, MassBasedTank, Fluid, CylindricalTank

In [ ]:
%matplotlib widget

# Input Curves

In [ ]:
LOX_Volume_Liters = Function("../../data/SEBLM/test124_Lox_Volume.csv",extrapolation="zero")
LOX_Volume = LOX_Volume_Liters * 0.001
LOX_Volume.setDiscrete(8.003, 19.984, 40,interpolation="linear")
LOX_Volume.plot(forceData=True)
LOX_tank_ullage = 0.013167926436231077 - LOX_Volume
LOX_tank_ullage.plot(8,8.5,forceData=True)

In [ ]:
Propane_Volume_Liters = Function("../../data/SEBLM/test124_Propane_Volume.csv")
Propane_Volume = Propane_Volume_Liters * 0.001 # m3
Propane_Volume.setDiscrete(8.003, 19.984, 40,"linear")
Propane_Volume.plot(forceData=True)
Propane_tank_ullage = 0.013167926436231077 - Propane_Volume
Propane_tank_ullage.plot(forceData=True)

# Fluids

In [ ]:
LOX = Fluid("LOX", 1024, 1)
Propane = Fluid("Propane", 566, 1)
LOXTankPressurizingGas = Fluid("N2", 31.3/28, 1) # 450 PSI
PropaneTankPressurizingGas = Fluid("N2", 313*300/4500/28, 1) # 300 PSI
PressurizingGas = Fluid("N2", 313/28, 1)# 4500 PSI

# LOX Tank

In [ ]:
LOX_tank_geometry = CylindricalTank(0.0744,0.658,spherical_caps=True)
LOX_tank = UllageBasedTank(
    name="LOX Tank",
    geometry=LOX_tank_geometry,
    gas=LOXTankPressurizingGas,
    liquid=LOX,
    ullage=LOX_tank_ullage,
)

In [ ]:
LOX_tank.allInfo()

# Propane Tank

In [ ]:
Propane_tank_geometry = CylindricalTank(0.0744,0.658,spherical_caps=True)
Propane_tank = UllageBasedTank(
    name="Propane Tank",
    geometry=Propane_tank_geometry,
    gas=PropaneTankPressurizingGas,
    liquid=Propane,
    ullage=Propane_tank_ullage,
)

In [ ]:
Propane_tank.allInfo()

# Pressure Tank

In [ ]:
Pressure_tank_geometry = CylindricalTank(0.135/2,0.846,spherical_caps=True)
Pressure_tank = MassBasedTank(
    name="Pressure Tank",
    geometry=Pressure_tank_geometry,
    liquid_mass=0,
    gas_mass="../../data/SEBLM/pressurantMassFiltered.csv",
    gas=PressurizingGas,
    liquid=PressurizingGas,
)


In [ ]:
Pressure_tank.geometry.total_volume

In [ ]:
Pressure_tank.liquidMass()

# Liquid Motor

In [ ]:
SEBLM = LiquidMotor(
    thrustSource="../../data/SEBLM/test124_Thrust_Curve.csv",
    burnOut=20.0,
    nozzleRadius=0.069/2,
    nozzlePosition=-1.364,
    coordinateSystemOrientation="nozzleToCombustionChamber",
)

SEBLM.addTank(Propane_tank, position=-1.048)
SEBLM.addTank(LOX_tank, position=0.711)
SEBLM.addTank(Pressure_tank, position=2.007)

In [ ]:
SEBLM.allInfo()

In [ ]:
SEBLM.mass.setDiscrete(8, 20, 50)
SEBLM.massDot.setDiscrete(8, 20, 50)
SEBLM.centerOfMass.setDiscrete(8, 20, 50)
SEBLM.Ixx.setDiscrete(8, 20, 50)
SEBLM.Iyy.setDiscrete(8, 20, 50)

In [ ]:
print(SEBLM.burnOutTime)
print(SEBLM._csys)
print(SEBLM.nozzlePosition)
SEBLM.mass.plot(8, 20)
SEBLM.massDot.plot(8, 20)
SEBLM.thrust.plot(8, 20)
SEBLM.centerOfMass.plot(8, 20)
SEBLM.Ixx.plot(8, 20)
SEBLM.Iyy.plot(8, 20)
SEBLM.Izz.plot(8, 20)
SEBLM.Ixy.plot(8, 20)
SEBLM.Ixz.plot(8, 20)
SEBLM.Iyz.plot(8, 20)

# Rocket Definition

In [ ]:
from rocketpy import Rocket

SEBRocket = Rocket(
    radius=0.098,
    mass=63.4,
    inertia=(25,25,1),
    powerOffDrag="../../data/SEBLM/drag.csv",
    powerOnDrag="../../data/SEBLM/drag.csv",
    centerOfDryMassPosition=3.23,
    coordinateSystemOrientation="noseToTail",
)
SEBRocket.addMotor(SEBLM, position=5.75)
SEBRocket.addNose(length=0.7, kind="vonKarman", position=0)
SEBRocket.addTail(
    topRadius=0.098, bottomRadius=0.058, length=0.198, position=5.69-0.198
)

SEBRocket.addTrapezoidalFins(
    n=4,
    rootChord=0.355,
    tipChord=0.0803,
    span=0.156,
    position=5.25,
    cantAngle=0,
)

# Ignore next line
SEBRocket.setRailButtons([-1, 1])


In [ ]:
SEBRocket.allInfo()

In [ ]:
from rocketpy import Environment

env = Environment(
    railLength=18.28, latitude=35.347122986338356, longitude=-117.80893423073582
)

# env.setDate((2022,12,3,14+7,0,0)) # UTC
# env.setAtmosphericModel(type="Forecast", file="GFS")

In [ ]:
from rocketpy import Flight

TestFlight = Flight(rocket=SEBRocket, environment=env, inclination=90, heading=23, maxTimeStep=0.1)

In [ ]:
TestFlight.angleOfAttack.plot(TestFlight.outOfRailTime, 15)

In [ ]:
TestFlight.allInfo()

- Rocket radius (largest radius)
- Rocket dry mass (includes everything except propellant)
- Rocket inertia tensor (I, Z)
- Drag curves (power on, power off)
- Nozzle position 
- NoseCone length, kind (von Karman) and position (tip position)
- FinSet (n=3, rootChord, tipChord, span, position (tip of root chord), cantAngle)
- Boattail top radius, bottom radius, length, position (top radius position)